# Preparación de Datos con PySpark

Este notebook realiza la preparación de datos usando PySpark para procesar documentos PDF.

In [3]:
# Configuración inicial
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import udf, col
import numpy as np
import subprocess

# Instalar pdf2image si no está disponible
try:
    import pdf2image
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pdf2image"])
finally:
    from pdf2image import convert_from_path

import cv2

# Configurar Spark
print("Configurando Spark...")
spark = SparkSession.builder \
    .appName("PDFPreprocessing") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()
print("Spark configurado correctamente.")

# Configurar directorios
RAW_DIR = 'data/raw'
PROCESSED_DIR = 'data/processed'

# Validar y crear directorios si no existen
if not os.path.exists(RAW_DIR):
    print(f"Directorio {RAW_DIR} no existe. Creándolo...")
    os.makedirs(RAW_DIR, exist_ok=True)
else:
    print(f"Directorio {RAW_DIR} ya existe.")

if not os.path.exists(PROCESSED_DIR):
    print(f"Directorio {PROCESSED_DIR} no existe. Creándolo...")
    os.makedirs(PROCESSED_DIR, exist_ok=True)
else:
    print(f"Directorio {PROCESSED_DIR} ya existe.")

print(f"Directorios configurados: RAW_DIR={RAW_DIR}, PROCESSED_DIR={PROCESSED_DIR}")

Configurando Spark...
Spark configurado correctamente.
Directorio data/raw ya existe.
Directorio data/processed ya existe.
Directorios configurados: RAW_DIR=data/raw, PROCESSED_DIR=data/processed


In [4]:
# Procesar PDFs a imágenes
def process_pdf(pdf_path):
    """Procesa un PDF y guarda sus páginas como imágenes"""
    try:
        print(f"Procesando {pdf_path}...")
        # Convertir PDF a imágenes
        pages = convert_from_path(pdf_path)
        pdf_name = os.path.basename(pdf_path).replace('.pdf', '')
        
        # Crear directorio para este PDF
        pdf_dir = os.path.join(PROCESSED_DIR, pdf_name)
        os.makedirs(pdf_dir, exist_ok=True)
        
        # Guardar cada página como imagen
        for i, page in enumerate(pages):
            page_path = os.path.join(pdf_dir, f'page_{i}.png')
            page.save(page_path)
            print(f"Guardada página {i} de {pdf_name} en {page_path}")
            
        print(f"Procesamiento de {pdf_name} completado.")
    except Exception as e:
        print(f"Error procesando {pdf_path}: {str(e)}")

# Procesar todos los PDFs en el directorio RAW_DIR
pdf_files = [os.path.join(RAW_DIR, f) for f in os.listdir(RAW_DIR) if f.endswith('.pdf')]
for pdf_file in pdf_files:
    process_pdf(pdf_file)

Procesando data/raw\2-TITULOS-15-DE-NOVIEMBRE-2024.pdf...
Guardada página 0 de 2-TITULOS-15-DE-NOVIEMBRE-2024 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024\page_0.png
Guardada página 1 de 2-TITULOS-15-DE-NOVIEMBRE-2024 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024\page_1.png
Guardada página 2 de 2-TITULOS-15-DE-NOVIEMBRE-2024 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024\page_2.png
Guardada página 3 de 2-TITULOS-15-DE-NOVIEMBRE-2024 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024\page_3.png
Guardada página 4 de 2-TITULOS-15-DE-NOVIEMBRE-2024 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024\page_4.png
Guardada página 5 de 2-TITULOS-15-DE-NOVIEMBRE-2024 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024\page_5.png
Guardada página 6 de 2-TITULOS-15-DE-NOVIEMBRE-2024 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024\page_6.png
Guardada página 7 de 2-TITULOS-15-DE-NOVIEMBRE-2024 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024\page_7.png
Guardada página 8 de 2-TITULOS-15-DE-NOVIEMBRE

In [4]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import json
import logging
import os

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class PDFLabeler:
    def __init__(self, processed_dir='data/processed', labels_file='data/labels.json'):
        self.processed_dir = processed_dir
        self.labels_file = labels_file
        self.window = tk.Tk()
        self.window.title("PDF Page Labeler")
        self.setup_gui()
        self.load_existing_labels()

    def setup_gui(self):
        """Configura la interfaz gráfica"""
        # Frame principal
        self.main_frame = ttk.Frame(self.window, padding="10")
        self.main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Área de visualización
        self.image_label = ttk.Label(self.main_frame)
        self.image_label.grid(row=0, column=0, rowspan=6, pady=5)

        # Controles
        ttk.Label(self.main_frame, text="PDF actual:").grid(
            row=0, column=1, sticky=tk.W)
        self.pdf_label = ttk.Label(self.main_frame, text="")
        self.pdf_label.grid(row=0, column=2, sticky=tk.W)

        ttk.Label(self.main_frame, text="Página:").grid(
            row=1, column=1, sticky=tk.W)
        self.page_label = ttk.Label(self.main_frame, text="")
        self.page_label.grid(row=1, column=2, sticky=tk.W)

        # Botones
        ttk.Button(self.main_frame, text="Es Primera Página",
                   command=self.mark_first_page).grid(row=2, column=1, pady=5)
        ttk.Button(self.main_frame, text="No es Primera Página",
                   command=self.next_page).grid(row=2, column=2, pady=5)
        ttk.Button(self.main_frame, text="Guardar y Salir",
                   command=self.save_and_exit).grid(row=3, column=1, columnspan=2, pady=5)
        
        # Agregar botones de navegación
        ttk.Button(self.main_frame, text="← Anterior",
                   command=self.previous_page).grid(row=4, column=1, pady=5)
        ttk.Button(self.main_frame, text="Siguiente →",
                   command=self.next_page).grid(row=4, column=2, pady=5)

        # Mostrar estado de etiquetado
        self.status_label = ttk.Label(self.main_frame, text="")
        self.status_label.grid(row=5, column=1, columnspan=2, pady=5)

        # Agregar atajos de teclado
        self.window.bind('<Left>', lambda e: self.previous_page())
        self.window.bind('<Right>', lambda e: self.next_page())
        self.window.bind('<space>', lambda e: self.mark_first_page())
        self.window.bind('q', lambda e: self.save_and_exit())

    def previous_page(self):
        """Retrocede a la página anterior"""
        if self.current_page > 0:
            self.current_page -= 1
            self.show_current_page()

    def show_current_page(self):
        """Muestra la página actual"""
        try:
            if self.current_pdf_index >= len(self.pdfs):
                return  # No intentar mostrar una página si ya hemos terminado

            page_path = os.path.join(
                self.processed_dir,
                self.current_pdf,
                f'page_{self.current_page}.png'
            )

            if not os.path.exists(page_path):
                raise FileNotFoundError(
                    f"No se encuentra la página: {page_path}")

            # Cargar y mostrar imagen
            image = Image.open(page_path)

            # Mantener proporción de aspecto
            width, height = image.size
            ratio = min(800/width, 600/height)
            new_size = (int(width * ratio), int(height * ratio))
            image = image.resize(new_size, Image.Resampling.LANCZOS)

            photo = ImageTk.PhotoImage(image)
            self.image_label.configure(image=photo)
            self.image_label.image = photo

            # Actualizar etiquetas
            self.pdf_label.configure(text=self.current_pdf)
            self.page_label.configure(
                text=f"Página {self.current_page + 1} de {self.get_total_pages()}")

            # Actualizar estado
            is_marked = self.is_page_marked(self.current_page)
            status = "MARCADA COMO PRIMERA PÁGINA" if is_marked else "NO MARCADA"
            self.status_label.configure(text=f"Estado: {status}")

        except Exception as e:
            logger.error(f"Error mostrando página: {e}")
            self.status_label.configure(text="Error mostrando la página")

    def get_total_pages(self):
        """Obtiene el total de páginas del PDF actual"""
        pdf_dir = os.path.join(self.processed_dir, self.current_pdf)
        return len([f for f in os.listdir(pdf_dir) if f.endswith('.png')])

    def is_page_marked(self, page_num):
        """Verifica si la página está marcada como primera página"""
        return (self.current_pdf in self.labels and
                page_num in self.labels[self.current_pdf]["target_pages"])

    def load_existing_labels(self):
        """Carga etiquetas existentes si las hay"""
        try:
            if os.path.exists(self.labels_file):
                with open(self.labels_file, 'r') as f:
                    self.labels = json.load(f)
            else:
                self.labels = {}
        except Exception as e:
            logger.error(f"Error cargando etiquetas: {e}")
            self.labels = {}

    def start_labeling(self):
        """Inicia el proceso de etiquetado"""
        pdfs = [d for d in os.listdir(self.processed_dir)
                if os.path.isdir(os.path.join(self.processed_dir, d))]

        if not pdfs:
            logger.error("No se encontraron PDFs procesados")
            return

        self.pdfs = pdfs
        self.current_pdf_index = 0
        self.current_pdf = self.pdfs[self.current_pdf_index]
        self.current_page = 0
        self.show_current_page()
        self.window.mainloop()

    def mark_first_page(self):
        """Marca la página actual como primera página"""
        if self.current_pdf not in self.labels:
            self.labels[self.current_pdf] = {
                "target_pages": [], "total_pages": 0}

        if self.current_page not in self.labels[self.current_pdf]["target_pages"]:
            self.labels[self.current_pdf]["target_pages"].append(
                self.current_page)
            logger.info(
                f"Página {self.current_page} marcada como primera página")

        self.next_page()

    def next_page(self):
        """Pasa a la siguiente página"""
        self.current_page += 1
        pages = os.listdir(os.path.join(self.processed_dir, self.current_pdf))

        if self.current_page >= len(pages):
            self.labels[self.current_pdf]["total_pages"] = len(pages)
            self.save_labels()
            self.load_next_pdf()
        else:
            self.show_current_page()

    def load_next_pdf(self):
        """Carga el siguiente PDF no etiquetado"""
        self.current_pdf_index += 1

        if self.current_pdf_index < len(self.pdfs):
            self.current_pdf = self.pdfs[self.current_pdf_index]
            self.current_page = 0
            self.show_current_page()
        else:
            logger.info("Todos los PDFs han sido etiquetados")
            self.save_and_exit()

    def save_labels(self):
        """Guarda las etiquetas en el archivo"""
        try:
            with open(self.labels_file, 'w') as f:
                json.dump(self.labels, f, indent=4)
            logger.info("Etiquetas guardadas correctamente")
        except Exception as e:
            logger.error(f"Error guardando etiquetas: {e}")

    def save_and_exit(self):
        """Guarda las etiquetas y cierra la aplicación"""
        self.save_labels()
        self.window.after(1000, self.window.quit)  # Esperar un segundo antes de cerrar

if __name__ == "__main__":
    labeler = PDFLabeler()
    labeler.start_labeling()

INFO:__main__:Página 110 marcada como primera página
INFO:__main__:Página 114 marcada como primera página
INFO:__main__:Página 146 marcada como primera página
INFO:__main__:Página 170 marcada como primera página
INFO:__main__:Página 178 marcada como primera página
INFO:__main__:Página 184 marcada como primera página
INFO:__main__:Página 190 marcada como primera página
INFO:__main__:Página 198 marcada como primera página
INFO:__main__:Página 206 marcada como primera página
INFO:__main__:Página 214 marcada como primera página
INFO:__main__:Página 222 marcada como primera página
INFO:__main__:Página 232 marcada como primera página
INFO:__main__:Página 244 marcada como primera página
INFO:__main__:Página 254 marcada como primera página
INFO:__main__:Página 264 marcada como primera página
INFO:__main__:Página 274 marcada como primera página
INFO:__main__:Página 284 marcada como primera página
INFO:__main__:Página 290 marcada como primera página
INFO:__main__:Página 296 marcada como primera 

: 

In [1]:
# Paso 4: Crear un DataFrame de Spark con las Imágenes y Etiquetas
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col
import cv2
import numpy as np

# Configurar Spark
print("Configurando Spark...")
spark = SparkSession.builder \
    .appName("ImageProcessing") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()
print("Spark configurado correctamente.")

PROCESSED_DIR = 'data/processed'
LABELS_JSON = 'data/labels.json'

def preprocess_image(image_path, target_size=(224, 224)):
    """Preprocesa una imagen para el modelo"""
    try:
        print(f"Preprocesando imagen {image_path}...")
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        resized = cv2.resize(image, target_size)
        normalized = resized.astype(np.float32) / 255.0
        return normalized.flatten().tolist()
    except Exception as e:
        print(f"Error preprocesando imagen {image_path}: {str(e)}")
        return None

# Leer etiquetas desde el archivo JSON
with open(LABELS_JSON, 'r') as f:
    labels = json.load(f)

# Crear DataFrame de imágenes y etiquetas
image_data = []
for pdf_name, pdf_info in labels.items():
    for page_num in range(pdf_info["total_pages"]):
        image_path = os.path.join(PROCESSED_DIR, pdf_name, f'page_{page_num}.png')
        label = 1 if page_num in pdf_info["target_pages"] else 0
        features = preprocess_image(image_path)
        if features is not None:
            image_data.append((image_path, label, features))

schema = StructType([
    StructField("path", StringType(), False),
    StructField("label", IntegerType(), False),
    StructField("features", ArrayType(FloatType()), True)
])

df = spark.createDataFrame(image_data, schema)
print("DataFrame creado correctamente.")

# Guardar DataFrame procesado
output_path = os.path.join(PROCESSED_DIR, "processed_images_with_labels.parquet")
print(f"Guardando DataFrame en {output_path}...")
df.write.mode("overwrite").parquet(output_path)
print("DataFrame guardado correctamente.")

# Mostrar estadísticas del procesamiento
print(f"Total de imágenes procesadas: {df.count()}")
print("\nDistribución de etiquetas:")
df.groupBy("label").count().show()

# Verificar valores nulos o problemas
print("\nVerificación de calidad:")
df.summary().show()

Configurando Spark...
Spark configurado correctamente.


NameError: name 'json' is not defined

In [3]:
import os
import sys
import json
import cv2
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler

# Configuración del entorno
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Configuración de rutas
PROCESSED_DIR = 'data/processed'
LABELS_JSON = 'data/labels.json'
MODEL_DIR = 'data/models'
os.makedirs(MODEL_DIR, exist_ok=True)

# Inicializar Spark
spark = SparkSession.builder \
    .appName("DocumentClassifier") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.sql.shuffle.partitions", "10") \
    .getOrCreate()

def preprocess_image(image_path, target_size=(224, 224)):
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f"No se pudo cargar la imagen: {image_path}")
        resized = cv2.resize(image, target_size)
        normalized = resized.astype(np.float32) / 255.0
        return normalized.flatten().tolist()
    except Exception as e:
        print(f"Error procesando {image_path}: {str(e)}")
        return None

# Cargar y procesar datos
with open(LABELS_JSON, 'r') as f:
    labels = json.load(f)

image_data = []
for pdf_name, pdf_info in labels.items():
    for page_num in range(pdf_info["total_pages"]):
        image_path = os.path.join(PROCESSED_DIR, pdf_name, f'page_{page_num}.png')
        if os.path.exists(image_path):
            label = 1 if page_num in pdf_info["target_pages"] else 0
            features = preprocess_image(image_path)
            if features is not None:
                image_data.append((image_path, label, features))

# Crear DataFrame
schema = StructType([
    StructField("path", StringType(), False),
    StructField("label", IntegerType(), False),
    StructField("features", ArrayType(FloatType()), True)
])

df = spark.createDataFrame(image_data, schema)

# Convertir features a vector
array_to_vector_udf = udf(lambda x: Vectors.dense(x), VectorUDT())
df = df.withColumn("features_vector", array_to_vector_udf("features"))

# Escalar características
scaler = StandardScaler(inputCol="features_vector", 
                       outputCol="scaled_features",
                       withStd=True,
                       withMean=True)
scaler_model = scaler.fit(df)
df_scaled = scaler_model.transform(df)

# Preparar datos para entrenamiento
final_df = df_scaled.select("label", "scaled_features")
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)

# Verificar datos antes de entrenar
print(f"Total de registros en el conjunto de entrenamiento: {train_df.count()}")
print(f"Total de registros en el conjunto de prueba: {test_df.count()}")

# Entrenar modelo
lr = LogisticRegression(
    featuresCol="scaled_features",
    labelCol="label",
    maxIter=20,
    regParam=0.1,
    elasticNetParam=0.8
)

try:
    lr_model = lr.fit(train_df)
except Exception as e:
    print(f"Error durante el entrenamiento del modelo: {str(e)}")
    spark.stop()
    sys.exit(1)

# Evaluar modelo
predictions = lr_model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

# Métricas
accuracy = evaluator.evaluate(predictions)
print(f"\nMétricas del modelo:")
print(f"Accuracy: {accuracy:.4f}")

# Guardar modelo y datos procesados
lr_model.save(os.path.join(MODEL_DIR, "logistic_regression_model"))
df_scaled.write.mode("overwrite").parquet(os.path.join(PROCESSED_DIR, "processed_scaled_data.parquet"))

# Mostrar estadísticas
print("\nEstadísticas del procesamiento:")
print(f"Total imágenes: {df.count()}")
print("\nDistribución de clases:")
df.groupBy("label").count().show()

spark.stop()

Total de registros en el conjunto de entrenamiento: 304
Total de registros en el conjunto de prueba: 66

Métricas del modelo:
Accuracy: 0.8939

Estadísticas del procesamiento:
Total imágenes: 370

Distribución de clases:
+-----+-----+
|label|count|
+-----+-----+
|    0|  325|
|    1|   45|
+-----+-----+



In [6]:
import os
import sys
import json
import cv2
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler
from pyspark.sql import functions as F

# Configuración del entorno
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Configuración de rutas
PROCESSED_DIR = 'data/processed'
LABELS_JSON = 'data/labels.json'
MODEL_DIR = 'data/models'
os.makedirs(MODEL_DIR, exist_ok=True)

# Inicializar Spark
spark = SparkSession.builder \
    .appName("DocumentClassifier") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.sql.shuffle.partitions", "10") \
    .getOrCreate()

def preprocess_image(image_path, target_size=(224, 224)):
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f"No se pudo cargar la imagen: {image_path}")
        resized = cv2.resize(image, target_size)
        normalized = resized.astype(np.float32) / 255.0
        return normalized.flatten().tolist()
    except Exception as e:
        print(f"Error procesando {image_path}: {str(e)}")
        return None

# Cargar y procesar datos
with open(LABELS_JSON, 'r') as f:
    labels = json.load(f)

image_data = []
for pdf_name, pdf_info in labels.items():
    for page_num in range(pdf_info["total_pages"]):
        image_path = os.path.join(PROCESSED_DIR, pdf_name, f'page_{page_num}.png')
        if os.path.exists(image_path):
            label = 1 if page_num in pdf_info["target_pages"] else 0
            features = preprocess_image(image_path)
            if features is not None:
                image_data.append((image_path, label, features))

# Crear DataFrame
schema = StructType([
    StructField("path", StringType(), False),
    StructField("label", IntegerType(), False),
    StructField("features", ArrayType(FloatType()), True)
])

df = spark.createDataFrame(image_data, schema)

# Convertir features a vector
array_to_vector_udf = udf(lambda x: Vectors.dense(x), VectorUDT())
df = df.withColumn("features_vector", array_to_vector_udf("features"))

# Escalar características
scaler = StandardScaler(inputCol="features_vector", 
                       outputCol="scaled_features",
                       withStd=True,
                       withMean=True)
scaler_model = scaler.fit(df)
df_scaled = scaler_model.transform(df)

# Sobremuestreo de la clase minoritaria
minority_class_df = df_scaled.filter(col("label") == 1)
majority_class_df = df_scaled.filter(col("label") == 0)

# Número de ejemplos en la clase mayoritaria
majority_count = majority_class_df.count()

# Sobremuestrear la clase minoritaria
oversampled_minority_class_df = minority_class_df.sample(withReplacement=True, fraction=majority_count / minority_class_df.count())

# Combinar los DataFrames
balanced_df = majority_class_df.union(oversampled_minority_class_df)

# Verificar la nueva distribución de clases
balanced_df.groupBy("label").count().show()

# Preparar datos para entrenamiento
final_df = balanced_df.select("label", "scaled_features")
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)

# Verificar datos antes de entrenar
print(f"Total de registros en el conjunto de entrenamiento: {train_df.count()}")
print(f"Total de registros en el conjunto de prueba: {test_df.count()}")

# Entrenar modelo
lr = LogisticRegression(
    featuresCol="scaled_features",
    labelCol="label",
    maxIter=20,
    regParam=0.1,
    elasticNetParam=0.8
)

try:
    lr_model = lr.fit(train_df)
except Exception as e:
    print(f"Error durante el entrenamiento del modelo: {str(e)}")
    spark.stop()
    sys.exit(1)

# Evaluar modelo
predictions = lr_model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

# Métricas
accuracy = evaluator.evaluate(predictions)
print(f"\nMétricas del modelo:")
print(f"Accuracy: {accuracy:.4f}")

# Guardar modelo y datos procesados
try:
    lr_model.write().overwrite().save(os.path.join(MODEL_DIR, "logistic_regression_model"))
except Exception as e:
    print(f"Error al guardar el modelo: {str(e)}")

try:
    df_scaled.write.mode("overwrite").parquet(os.path.join(PROCESSED_DIR, "processed_scaled_data.parquet"))
except Exception as e:
    print(f"Error al guardar los datos procesados: {str(e)}")

# Mostrar estadísticas
print("\nEstadísticas del procesamiento:")
print(f"Total imágenes: {df.count()}")
print("\nDistribución de clases:")
df.groupBy("label").count().show()

spark.stop()

+-----+-----+
|label|count|
+-----+-----+
|    0|  325|
|    1|  325|
+-----+-----+

Total de registros en el conjunto de entrenamiento: 521
Total de registros en el conjunto de prueba: 129

Métricas del modelo:
Accuracy: 0.9612

Estadísticas del procesamiento:
Total imágenes: 370

Distribución de clases:
+-----+-----+
|label|count|
+-----+-----+
|    0|  325|
|    1|   45|
+-----+-----+



In [1]:
import os
import sys
import json
import cv2
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler
from pyspark.sql import functions as F

# Configuración del entorno
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Configuración de rutas
PROCESSED_DIR = 'data/processed'
LABELS_JSON = 'data/labels.json'
MODEL_DIR = 'data/models'
os.makedirs(MODEL_DIR, exist_ok=True)

# Inicializar Spark
spark = SparkSession.builder \
    .appName("DocumentClassifier") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.sql.shuffle.partitions", "10") \
    .getOrCreate()

def preprocess_image(image_path, target_size=(224, 224)):
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f"No se pudo cargar la imagen: {image_path}")
        resized = cv2.resize(image, target_size)
        normalized = resized.astype(np.float32) / 255.0
        return normalized.flatten().tolist()
    except Exception as e:
        print(f"Error procesando {image_path}: {str(e)}")
        return None

# Cargar y procesar datos
with open(LABELS_JSON, 'r') as f:
    labels = json.load(f)

image_data = []
for pdf_name, pdf_info in labels.items():
    for page_num in range(pdf_info["total_pages"]):
        image_path = os.path.join(PROCESSED_DIR, pdf_name, f'page_{page_num}.png')
        if os.path.exists(image_path):
            label = 1 if page_num in pdf_info["target_pages"] else 0
            features = preprocess_image(image_path)
            if features is not None:
                image_data.append((image_path, label, features))

# Crear DataFrame
schema = StructType([
    StructField("path", StringType(), False),
    StructField("label", IntegerType(), False),
    StructField("features", ArrayType(FloatType()), True)
])

df = spark.createDataFrame(image_data, schema)

# Convertir features a vector
array_to_vector_udf = udf(lambda x: Vectors.dense(x), VectorUDT())
df = df.withColumn("features_vector", array_to_vector_udf("features"))

# Escalar características
scaler = StandardScaler(inputCol="features_vector", 
                       outputCol="scaled_features",
                       withStd=True,
                       withMean=True)
scaler_model = scaler.fit(df)
df_scaled = scaler_model.transform(df)

# Guardar el escalador
try:
    scaler_model.write().overwrite().save(os.path.join(PROCESSED_DIR, "scaler_model"))
except Exception as e:
    print(f"Error al guardar el escalador: {str(e)}")

# Sobremuestreo de la clase minoritaria
minority_class_df = df_scaled.filter(col("label") == 1)
majority_class_df = df_scaled.filter(col("label") == 0)

# Número de ejemplos en la clase mayoritaria
majority_count = majority_class_df.count()

# Sobremuestrear la clase minoritaria
oversampled_minority_class_df = minority_class_df.sample(withReplacement=True, fraction=majority_count / minority_class_df.count())

# Combinar los DataFrames
balanced_df = majority_class_df.union(oversampled_minority_class_df)

# Verificar la nueva distribución de clases
balanced_df.groupBy("label").count().show()

# Preparar datos para entrenamiento
final_df = balanced_df.select("label", "scaled_features")
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)

# Verificar datos antes de entrenar
print(f"Total de registros en el conjunto de entrenamiento: {train_df.count()}")
print(f"Total de registros en el conjunto de prueba: {test_df.count()}")

# Entrenar modelo
lr = LogisticRegression(
    featuresCol="scaled_features",
    labelCol="label",
    maxIter=20,
    regParam=0.1,
    elasticNetParam=0.8
)

try:
    lr_model = lr.fit(train_df)
except Exception as e:
    print(f"Error durante el entrenamiento del modelo: {str(e)}")
    spark.stop()
    sys.exit(1)

# Evaluar modelo
predictions = lr_model.transform(test_df)

# Evaluar con métricas adicionales
evaluator_accuracy = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)
evaluator_precision = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="weightedPrecision"
)
evaluator_recall = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="weightedRecall"
)
evaluator_f1 = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="f1"
)

accuracy = evaluator_accuracy.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)
f1 = evaluator_f1.evaluate(predictions)

print(f"\nMétricas del modelo:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Guardar modelo y datos procesados
try:
    lr_model.write().overwrite().save(os.path.join(MODEL_DIR, "logistic_regression_model"))
except Exception as e:
    print(f"Error al guardar el modelo: {str(e)}")

try:
    df_scaled.write.mode("overwrite").parquet(os.path.join(PROCESSED_DIR, "processed_scaled_data.parquet"))
except Exception as e:
    print(f"Error al guardar los datos procesados: {str(e)}")

# Mostrar estadísticas
print("\nEstadísticas del procesamiento:")
print(f"Total imágenes: {df.count()}")
print("\nDistribución de clases:")
df.groupBy("label").count().show()

spark.stop()

+-----+-----+
|label|count|
+-----+-----+
|    0|  325|
|    1|  346|
+-----+-----+

Total de registros en el conjunto de entrenamiento: 540
Total de registros en el conjunto de prueba: 131

Métricas del modelo:
Accuracy: 0.9466
Precision: 0.9496
Recall: 0.9466
F1-score: 0.9467

Estadísticas del procesamiento:
Total imágenes: 370

Distribución de clases:
+-----+-----+
|label|count|
+-----+-----+
|    0|  325|
|    1|   45|
+-----+-----+



In [9]:
!python run.py predict

Todas las bibliotecas necesarias de Hadoop están presentes.
CORRECTO: el proceso con PID 23144 (proceso secundario de PID 16040)
ha sido terminado.
CORRECTO: el proceso con PID 16040 (proceso secundario de PID 3188)
ha sido terminado.
CORRECTO: el proceso con PID 3188 (proceso secundario de PID 23072)
ha sido terminado.


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/04 23:57:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Traceback (most recent call last):
  File "e:\proyectos\validarpdf\document_classifier\notebooks\nuevos\scripts\predict_first_pages.py", line 122, in <module>
    predict_first_pages(pdf_dir, model_path, scaler_path)
  File "e:\proyectos\validarpdf\document_classifier\notebooks\nuevos\scripts\predict_first_pages.py", line 77, in predict_first_pages
    lr_model = LogisticRegressionModel.load(model_path)
  File "e:\proyectos\validarpdf\document_classifier\.venv\lib\site-packages\pyspark\ml\util.py", line 369, in load
    return cls.read().load(path)
  File "e:\proyectos\validarpdf\document_classifier\.venv\lib\site-packages\pyspark\ml\util.py", line 318, in load
    java_obj = self._jread.load(path)
  File "e:\proyectos\validarpdf\document_classifier\.venv\lib\s

In [7]:
import os
import sys
import ctypes
import cv2
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.feature import StandardScalerModel

def check_hadoop_libraries():
    # Verificar las variables de entorno
    hadoop_home = os.environ.get('HADOOP_HOME')
    java_home = os.environ.get('JAVA_HOME')
    
    if not hadoop_home:
        print("Error: HADOOP_HOME no está configurado.")
        return False
    
    if not java_home:
        print("Error: JAVA_HOME no está configurado.")
        return False
    
    # Verificar la existencia de winutils.exe
    winutils_path = os.path.join(hadoop_home, 'bin', 'winutils.exe')
    if not os.path.exists(winutils_path):
        print(f"Error: {winutils_path} no existe.")
        return False
    
    # Verificar la existencia de las bibliotecas nativas de Hadoop
    native_lib_path = os.path.join(hadoop_home, 'bin')
    try:
        ctypes.cdll.LoadLibrary(os.path.join(native_lib_path, 'hadoop.dll'))
    except OSError as e:
        print(f"Error: No se pudo cargar la biblioteca nativa de Hadoop: {e}")
        return False
    
    print("Todas las bibliotecas necesarias de Hadoop están presentes.")
    return True

# Verificar las bibliotecas necesarias de Hadoop antes de ejecutar el script
if not check_hadoop_libraries():
    sys.exit(1)

# Configuración del entorno
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Configurar las bibliotecas nativas de Hadoop
os.environ['HADOOP_HOME'] = 'C:\\Program Files\\winutils'
os.environ['JAVA_HOME'] = 'C:\\Program Files\\Java\\jdk-21'
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['HADOOP_HOME'], 'bin')
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['JAVA_HOME'], 'bin')

# Inicializar Spark
spark = SparkSession.builder \
    .appName("PDFFirstPagePredictor") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.sql.shuffle.partitions", "10") \
    .config("spark.executor.extraJavaOptions", "-Djava.library.path=" + os.path.join(os.environ['HADOOP_HOME'], 'bin')) \
    .getOrCreate()

def predict_first_pages(pdf_dir, model_path, scaler_path):
    # Verificar si el modelo y el escalador existen
    if not os.path.exists(model_path):
        print(f"Error: El modelo no existe en la ruta {model_path}")
        return
    if not os.path.exists(scaler_path):
        print(f"Error: El escalador no existe en la ruta {scaler_path}")
        return

    # Cargar el modelo y el escalador
    lr_model = LogisticRegressionModel.load(model_path)
    scaler_model = StandardScalerModel.load(scaler_path)

    # Procesar las páginas del PDF
    image_data = []
    for page_num in range(len(os.listdir(pdf_dir))):
        image_path = os.path.join(pdf_dir, f'page_{page_num}.png')
        if os.path.exists(image_path):
            # Leer la imagen y convertirla a un vector
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is not None:
                resized = cv2.resize(image, (224, 224))
                normalized = resized.astype(np.float32) / 255.0
                features = normalized.flatten().tolist()
                image_data.append((image_path, features))

    # Crear DataFrame
    schema = StructType([
        StructField("path", StringType(), False),
        StructField("features", ArrayType(FloatType()), True)
    ])
    df = spark.createDataFrame(image_data, schema)

    # Convertir features a vector
    array_to_vector_udf = udf(lambda x: Vectors.dense(x), VectorUDT())
    df = df.withColumn("features_vector", array_to_vector_udf("features"))

    # Escalar características
    df_scaled = scaler_model.transform(df)

    # Realizar predicciones
    predictions = lr_model.transform(df_scaled)

    # Filtrar las primeras páginas predichas
    first_pages = predictions.filter(col("prediction") == 1).select("path").collect()

    # Mostrar resultados
    print("Páginas predichas como primeras páginas:")
    for row in first_pages:
        print(row["path"])

# Ejemplo de uso
pdf_dir = 'data/pdf_pages'
model_path = 'data/models/logistic_regression_model'
scaler_path = 'data/processed/scaler_model'
predict_first_pages(pdf_dir, model_path, scaler_path)

spark.stop()

Todas las bibliotecas necesarias de Hadoop están presentes.


Py4JJavaError: An error occurred while calling o549.load.
: java.lang.UnsatisfiedLinkError: 'org.apache.hadoop.io.nativeio.NativeIO$POSIX$Stat org.apache.hadoop.io.nativeio.NativeIO$POSIX.stat(java.lang.String)'
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.stat(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.getStat(NativeIO.java:608)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.loadPermissionInfoByNativeIO(RawLocalFileSystem.java:934)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.loadPermissionInfo(RawLocalFileSystem.java:848)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.getPermission(RawLocalFileSystem.java:816)
	at org.apache.hadoop.fs.LocatedFileStatus.<init>(LocatedFileStatus.java:52)
	at org.apache.hadoop.fs.FileSystem$4.next(FileSystem.java:2199)
	at org.apache.hadoop.fs.FileSystem$4.next(FileSystem.java:2179)
	at org.apache.hadoop.mapred.FileInputFormat.singleThreaded

In [8]:
import os
import sys
import ctypes

def check_hadoop_libraries():
    # Verificar las variables de entorno
    hadoop_home = os.environ.get('HADOOP_HOME')
    java_home = os.environ.get('JAVA_HOME')
    
    if not hadoop_home:
        print("Error: HADOOP_HOME no está configurado.")
        return False
    
    if not java_home:
        print("Error: JAVA_HOME no está configurado.")
        return False
    
    # Verificar la existencia de winutils.exe
    winutils_path = os.path.join(hadoop_home, 'bin', 'winutils.exe')
    if not os.path.exists(winutils_path):
        print(f"Error: {winutils_path} no existe.")
        return False
    
    # Verificar la existencia de las bibliotecas nativas de Hadoop
    native_lib_path = os.path.join(hadoop_home, 'bin')
    try:
        ctypes.cdll.LoadLibrary(os.path.join(native_lib_path, 'hadoop.dll'))
    except OSError as e:
        print(f"Error: No se pudo cargar la biblioteca nativa de Hadoop: {e}")
        return False
    
    print("Todas las bibliotecas necesarias de Hadoop están presentes.")
    return True

# Verificar las bibliotecas necesarias de Hadoop antes de ejecutar el script
if not check_hadoop_libraries():
    sys.exit(1)

# Configuración del entorno
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Configurar las bibliotecas nativas de Hadoop
os.environ['HADOOP_HOME'] = 'C:\\Program Files\\winutils'
os.environ['JAVA_HOME'] = 'C:\\Program Files\\Java\\jdk-21'
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['HADOOP_HOME'], 'bin')
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['JAVA_HOME'], 'bin')

# Inicializar Spark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.feature import StandardScalerModel

spark = SparkSession.builder \
    .appName("PDFFirstPagePredictor") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.sql.shuffle.partitions", "10") \
    .config("spark.executor.extraJavaOptions", "-Djava.library.path=" + os.path.join(os.environ['HADOOP_HOME'], 'bin')) \
    .getOrCreate()

def predict_first_pages(pdf_dir, model_path, scaler_path):
    # Verificar si el modelo y el escalador existen
    if not os.path.exists(model_path):
        print(f"Error: El modelo no existe en la ruta {model_path}")
        return
    if not os.path.exists(scaler_path):
        print(f"Error: El escalador no existe en la ruta {scaler_path}")
        return

    # Cargar el modelo y el escalador
    lr_model = LogisticRegressionModel.load(model_path)
    scaler_model = StandardScalerModel.load(scaler_path)

    # Procesar las páginas del PDF
    image_data = []
    for page_num in range(len(os.listdir(pdf_dir))):
        image_path = os.path.join(pdf_dir, f'page_{page_num}.png')
        if os.path.exists(image_path):
            # Leer la imagen y convertirla a un vector
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is not None:
                resized = cv2.resize(image, (224, 224))
                normalized = resized.astype(np.float32) / 255.0
                features = normalized.flatten().tolist()
                image_data.append((image_path, features))

    # Crear DataFrame
    schema = StructType([
        StructField("path", StringType(), False),
        StructField("features", ArrayType(FloatType()), True)
    ])
    df = spark.createDataFrame(image_data, schema)

    # Convertir features a vector
    array_to_vector_udf = udf(lambda x: Vectors.dense(x), VectorUDT())
    df = df.withColumn("features_vector", array_to_vector_udf("features"))

    # Escalar características
    df_scaled = scaler_model.transform(df)

    # Realizar predicciones
    predictions = lr_model.transform(df_scaled)

    # Filtrar las primeras páginas predichas
    first_pages = predictions.filter(col("prediction") == 1).select("path").collect()

    # Mostrar resultados
    print("Páginas predichas como primeras páginas:")
    for row in first_pages:
        print(row["path"])

# Ejemplo de uso
pdf_dir = 'data/pdf_pages'
model_path = 'data/models/logistic_regression_model'
scaler_path = 'data/processed/scaler_model'
predict_first_pages(pdf_dir, model_path, scaler_path)

spark.stop()

Todas las bibliotecas necesarias de Hadoop están presentes.


Py4JJavaError: An error occurred while calling o553.load.
: java.lang.UnsatisfiedLinkError: 'org.apache.hadoop.io.nativeio.NativeIO$POSIX$Stat org.apache.hadoop.io.nativeio.NativeIO$POSIX.stat(java.lang.String)'
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.stat(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.getStat(NativeIO.java:608)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.loadPermissionInfoByNativeIO(RawLocalFileSystem.java:934)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.loadPermissionInfo(RawLocalFileSystem.java:848)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.getPermission(RawLocalFileSystem.java:816)
	at org.apache.hadoop.fs.LocatedFileStatus.<init>(LocatedFileStatus.java:52)
	at org.apache.hadoop.fs.FileSystem$4.next(FileSystem.java:2199)
	at org.apache.hadoop.fs.FileSystem$4.next(FileSystem.java:2179)
	at org.apache.hadoop.mapred.FileInputFormat.singleThreaded

In [13]:
# %%
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("CheckHadoopVersion").getOrCreate()

# Obtener la versión de Hadoop
hadoop_version = spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion()
print("Hadoop version:", hadoop_version)

# Detener la sesión de Spark
spark.stop()

# %%
import sys
print("Python version:", sys.version)

# %%
import pyspark
print("Spark version:", pyspark.__version__)

Hadoop version: 3.3.4
Python version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
Spark version: 3.5.4


In [15]:
# Verificar la Ruta de Hadoop y Java
import os

# Verificar la variable de entorno HADOOP_HOME
hadoop_home = os.environ.get('HADOOP_HOME')
if hadoop_home:
    print(f"HADOOP_HOME: {hadoop_home}")
else:
    print("HADOOP_HOME no está configurado.")

# Verificar la variable de entorno JAVA_HOME
java_home = os.environ.get('JAVA_HOME')
if java_home:
    print(f"JAVA_HOME: {java_home}")
else:
    print("JAVA_HOME no está configurado.")

# Verificar la ruta de Hadoop en el PATH
path = os.environ.get('PATH')
if path:
    print("Rutas en PATH:")
    for p in path.split(os.pathsep):
        print(p)
else:
    print("PATH no está configurado.")

# Verificar la Configuración de Spark
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("CheckHadoopConfig").getOrCreate()

# Verificar la configuración de Spark
spark_conf = spark.sparkContext.getConf().getAll()
print("\nConfiguración de Spark:")
for conf in spark_conf:
    print(conf)

# Detener la sesión de Spark
spark.stop()

HADOOP_HOME: C:\Program Files\winutils
JAVA_HOME: C:\Program Files\Java\jdk-21
Rutas en PATH:
E:\proyectos\validarpdf\document_classifier\.venv\Lib\site-packages\cv2\../../x64/vc14/bin
e:\proyectos\validarpdf\document_classifier\.venv\Scripts
C:\Program Files\Common Files\Oracle\Java\javapath
C:\Program Files\NVIDIA\CUDNN\v9.5\bin
C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\bin
C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\libnvvp



C:\Program Files\Python312\Scripts\
C:\Program Files\Python312\
C:\WINDOWS\system32
C:\WINDOWS
C:\WINDOWS\System32\Wbem
C:\WINDOWS\System32\WindowsPowerShell\v1.0\
C:\WINDOWS\System32\OpenSSH\
C:\Program Files\Git\cmd
C:\Program Files\nodejs\
C:\ProgramData\chocolatey\bin
C:\Program Files\Microsoft SQL Server\Client SDK\ODBC\170\Tools\Binn\
C:\Program Files (x86)\Microsoft SQL Server\150\Tools\Binn\
C:\Program Files\Microsoft SQL Server\150\Tools\Binn\
C:\Program Files\Microsoft SQL Server\150\DTS\Binn\
C:\Program Files (x86)\Micros

In [6]:
!pip freeze > requirements.txt

In [ ]:
!python run.py predict

CORRECTO: el proceso con PID 6960 (proceso secundario de PID 17960)
ha sido terminado.
CORRECTO: el proceso con PID 17960 (proceso secundario de PID 21840)
ha sido terminado.
CORRECTO: el proceso con PID 21840 (proceso secundario de PID 4560)
ha sido terminado.


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Traceback (most recent call last):
  File "e:\proyectos\validarpdf\document_classifier\notebooks\nuevos\scripts\predict_first_pages.py", line 80, in <module>
    predict_first_pages(pdf_dir, model_path, scaler_path)
  File "e:\proyectos\validarpdf\document_classifier\notebooks\nuevos\scripts\predict_first_pages.py", line 39, in predict_first_pages
    lr_model = LogisticRegressionModel.load(model_path)
  File "e:\proyectos\validarpdf\document_classifier\.venv\lib\site-packages\pyspark\ml\util.py", line 369, in load
    return cls.read().load(path)
  File "e:\proyectos\validarpdf\document_classifier\.venv\lib\site-packages\pyspark\ml\util.py", line 318, in load
    java_obj = self._jread.load(path)
  File "e:\proyectos\validarpdf\document_classifier\.venv\lib\site-packages\py4j\java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  Fi

In [5]:
# echo $SPARK_HOME  # En Linux/Mac
!echo %SPARK_HOME% 
# En Windows

C:\Program Files\spark-3.5.4-bin-hadoop3


In [8]:
!python -c "import numpy"

In [6]:
!pip install numpy
import numpy as np
print(np.__version__)

2.0.2
